# Make efficiency map plots for trained fitters

In [1]:
import train as tn
reload(tn)

import plotting
reload(plotting)

import util as ut

import matplotlib.pyplot as plt
#plt.style.use('seaborn-ticks')
plt.style.use('seaborn-poster')
%matplotlib inline
#%matplotlib notebook

import numpy as np

from pprint import pprint

import itertools

# import xgboost

Welcome to ROOTaaS 6.06/08


In [2]:
#note that the loading option is True by default
ut.defaultParameters(dataDir='./data', inputName="fullBDT", inputDir='./classifiers')
ut.setParams()
effFitter = ut.loadOrMake()

entered config files named my_train_config
None
Load object with the name fullBDT and the following paramters 
./classifiers
./classifiers/fullBDT.pkl.gz
loading pickle ./classifiers/fullBDT.pkl.gz
loading data ./classifiers/fullBDT.root
Index([u'absweight', u'class', u'genJet2p5Pt0', u'genJet2p5Pt1',
       u'genJet2p5Pt2', u'genJet2p5Pt3', u'genJet2p5Rapidity0',
       u'genJet2p5Rapidity1', u'genJet2p5Rapidity2', u'genJet2p5Rapidity3',
       u'genNjets2p5', u'genPt', u'genRapidity', u'recoNjets2p5', u'recoPt',
       u'recoRapidity', u'weight', u'proc', u'absGenRapidity',
       u'absGenJet2p5Rapidity0', u'absGenJet2p5Rapidity1',
       u'absGenJet2p5Rapidity2', u'absGenJet2p5Rapidity3', u'recoPtBin',
       u'recoPtCat', u'recoNjets2p5Bin', u'recoNjets2p5Cat', u'class_prob_0',
       u'class_prob_1', u'class_prob_2', u'class_prob_3', u'recoPtCat_prob_0',
       u'recoPtCat_prob_1', u'recoPtCat_prob_2', u'recoPtCat_prob_3',
       u'recoPtCat_prob_4', u'recoPtCat_prob_5', u'recoPtC

## projection on the following 2 variables

In [12]:
x_var = 'absGenJet2p5Rapidity1'
y_var = 'genJet2p5Pt2'

In [42]:
def GetBins (x_pt, y_pt, x_name, y_name) :
    N_PtBins = 8
    N_RapidityBins = 8
    if (x_pt and y_pt) :
        return { x_name : dict(boundaries=np.linspace(0,300.,N_PtBins)), 
                       y_name : dict(boundaries=np.linspace(0,300.,N_PtBins))
            }   
    else :
        if x_pt :
            return { x_name : dict(boundaries=np.linspace(0,300.,N_PtBins)), 
                           y_name : dict(boundaries=np.linspace(0.,2.5,N_RapidityBins))
                          }
        if y_pt :
            return { x_name : dict(boundaries=np.linspace(.0,2.5,N_RapidityBins)), 
                           y_name : dict(boundaries=np.linspace(0,300.,N_PtBins))
                          }
    return { x_name : dict(boundaries=np.linspace(0.,2.5,N_RapidityBins)), 
             y_name : dict(boundaries=np.linspace(0.,2.5,N_RapidityBins))
                        }

### define bins in genJet2p5Rapidity0 and genJet2p5Pt0

In [43]:
defineBins = GetBins(x_pt=False,y_pt=True,x_name=x_var,y_name=y_var)

In [44]:
ut.runDefineBins(effFitter,defineBins)

genJet2p5Pt2
appending overflow
[   0.           42.85714286   85.71428571  128.57142857  171.42857143
  214.28571429  257.14285714  300.          577.99298096]
absGenJet2p5Rapidity1
[ 0.          0.35714286  0.71428571  1.07142857  1.42857143  1.78571429
  2.14285714  2.5       ]


In [45]:
print(defineBins)

{'genJet2p5Pt2': {'boundaries': array([   0.        ,   42.85714286,   85.71428571,  128.57142857,
        171.42857143,  214.28571429,  257.14285714,  300.        ])}, 'absGenJet2p5Rapidity1': {'boundaries': array([ 0.        ,  0.35714286,  0.71428571,  1.07142857,  1.42857143,
        1.78571429,  2.14285714,  2.5       ])}}


### Extract data frame and start getting the probas in a binned way

In [46]:
df = effFitter.df
first_train_evt = int(round(df.index.size*(1.-effFitter.split_frac)))

#take the test sample 
df_test = df[:first_train_evt]

In [47]:
#df_test[df_test['genJet2p5Pt0']>300][["genJet2p5Pt0Bin",'genJet2p5Pt0']]

In [48]:
df_test[['absGenJet2p5Rapidity1Bin','absGenJet2p5Rapidity1']][0:6]

,absGenJet2p5Rapidity1Bin,absGenJet2p5Rapidity1
random_index,,
0,NaN,0.0
1,NaN,0.0
2,NaN,0.0
3,NaN,0.0
4,NaN,0.0
5,NaN,0.0


In [53]:
df_test[['genJet2p5Pt2Bin','genJet2p5Pt2']][0:6]

,genJet2p5Pt2Bin,genJet2p5Pt2
random_index,,
0,NaN,0.0
1,NaN,0.0
2,NaN,0.0
3,NaN,0.0
4,NaN,0.0
5,NaN,0.0


In [10]:
def weighted_average(df_name, column_name, weight_name=None):
    """
    This function computes the weighted average of the quantity column_name
    stared in the pandas dataframe df_name. In case no weights are given
    or if they sum up to zero, the mean is returned instead.
    :params 
            df_name :
        column_name :
        weight_name :
    :retruns
                    :
    """
    d = df_name[column_name]
    w = df_name[weight_name]
    #print(len(w))
    if (weight_name == None) :
        return float(d.mean())
    else :
        try:
            return (d * w).sum() / float(w.sum())
        except ZeroDivisionError:
            return float(d.mean())

In [11]:
def weight_freq (df_name, column_name, equal_to, weight_name) :
    df = df_name#[df_name[column_name]==equal_to]
    
    w_all = df[weight_name].sum()
    w_PartPhaseSpace = df[df[column_name] == equal_to][weight_name].sum()
    #print(w_all)
    #print(w_PartPhaseSpace)
    return w_PartPhaseSpace / w_all

In [12]:
def plotRelDiff_imshow(groupby_objects,binBoundaries, 
                        x_lab = None,
                        y_lab = None,
                          cmap=plt.cm.Blues,
                      effPlot=False):
   
        
    fig = plt.figure(figsize=(7,7))

    r_pred, r_true = groupby_objects

    x_pred,y_pred = r_pred.index.levels
    x_true,y_true = r_true.index.levels

    cm_pred = r_pred.values.reshape(len(x_pred),len(y_pred))
    cm_true = r_true.values.reshape(len(x_true),len(y_true))

    if effPlot :
        cm_true = 1. - cm_true
        cm_pred = 1. - cm_pred
    
    #plt.subplot(111)
    total_sum_true_permille = np.sum(np.sum(cm_true))/100.
    print(total_sum_true_permille)
    
    cm = np.divide((cm_true-cm_pred),cm_true, out=np.zeros_like(cm_true-cm_pred), 
                   where=cm_true>total_sum_true_permille ) 
    
    cm = cm.T
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap,origin='lower')
    plt.title(r'$\frac{\mathrm{true} - \mathrm{pred}}{\mathrm{true}}$'+ '\n')
    
    
    
    xtick_marks = np.arange(len(x_pred)+1)-0.5
    xtick_names = binBoundaries[x_lab]['boundaries']
    ytick_marks = np.arange(len(y_pred))-0.5
    ytick_names = binBoundaries[y_lab]['boundaries']

    xtick_names = np.round(xtick_names,2)
    ytick_names = np.round(ytick_names,2)
    plt.xticks(xtick_marks,xtick_names,rotation='90')

    plt.xticks(xtick_marks, xtick_names)
    plt.yticks(ytick_marks, ytick_names)


    thresh = (cm.max()+cm.min()) / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, np.round(cm[i, j],2), horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")


    plt.xlabel(x_lab)
    plt.ylabel(y_lab)
    plt.tight_layout()

    fig.subplots_adjust(top=0.78)   
    cbar_ax = fig.add_axes([0.15, 0.97, 0.7, 0.02])
    cb = plt.colorbar(cax=cbar_ax,orientation='horizontal')#,norm=plt.colors.Normalize(vmin=min_col,vmax=max_col))

    #tick_locator = ticker.MaxNLocator(nbins=6)
    #cb.locator = tick_locator
    #cb.update_ticks()
       
    

In [13]:
def plot_imshow(groupby_objects, binBoundaries, 
                          titles=[],
                        x_lab = None,
                        y_lab = None,
                          cmap=plt.cm.Blues,
                   effTag=False, effPlot=False):
   

    
    if effTag :
        plot_imshow(groupby_objects, binBoundaries=binBoundaries, titles=['predicted (clf) reco eff', 'true (data) reco eff'],
                        x_lab = x_lab, y_lab = y_lab, cmap=plt.cm.Reds, effTag=False, effPlot=True)
    else :
        
        cm_list = []

        for k, r in enumerate(groupby_objects) :
            x,y = r.index.levels
            print(len(x))
            print(len(y))
            print(np.shape(r.values))
            cm = r.values.reshape(len(x),len(y))
            if effPlot :
                cm = 1. - cm
            
            #check this
            cm_list.append(cm.T)    
                        
        
        fig, axarr = plt.subplots(1,2,figsize=(10,10))
        
        
        minimum = np.amin(cm_list)
        maximum = np.amax(cm_list)
        
        for l, cm in enumerate(cm_list) :
        
            plt.subplot('22'+str(l+1))
            
            
            plt.imshow(cm_list[l], interpolation='nearest', cmap=cmap,origin='lower',
                       vmin=minimum,vmax=maximum)
            
            plt.title(titles[l])

            xtick_marks = np.arange(len(x)+1)-0.5
            xtick_names = binBoundaries[x_lab]['boundaries']
            ytick_marks = np.arange(len(y))-0.5
            ytick_names = binBoundaries[y_lab]['boundaries']
            
            xtick_names = np.round(xtick_names,2)
            ytick_names = np.round(ytick_names,2)
            plt.xticks(xtick_marks,xtick_names,rotation='90')
            
            
            
            

            plt.xlabel(x_lab)
            if l == 0 :
                print('hi')
                plt.ylabel(y_lab)
                plt.yticks(ytick_marks, ytick_names)
            else :
                #bad trick
                plt.yticks(visible=False)
            
            #plt.tight_layout()

            # text
            thresh = (maximum + minimum) / 2.
            for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
                plt.text(j, i, '%.2f' % cm[i, j], horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        
        fig.subplots_adjust(top=0.85)   
        cbar_ax = fig.add_axes([0.1, 0.98, 0.8, 0.02])
        plt.colorbar(cax=cbar_ax,orientation='horizontal')#,norm=plt.colors.Normalize(vmin=min_col,vmax=max_col))
        
        plt.show()
        if l == 0 :
            plotRelDiff_imshow(groupby_objects=groupby_objects,binBoundaries=binBoundaries,x_lab=x_var,y_lab=y_var,cmap=plt.cm.Oranges,effPlot=True)
        else :
            plotRelDiff_imshow(groupby_objects=groupby_objects,binBoundaries=binBoundaries,x_lab=x_var,y_lab=y_var,cmap=plt.cm.Oranges)

                 
    

In [15]:
N_recoNjets2p5Cats = 1

for i in np.arange(N_recoNjets2p5Cats) :
    column_proba_name = 'recoNjets2p5Cat_prob_'+str(i)
    
    gb_freq  = df.groupby([x_var+'Bin',y_var+'Bin']).apply(weight_freq,"recoNjets2p5Cat",(i-1),'absweight')
    gb_proba = df_test.groupby([x_var+'Bin',y_var+'Bin']).apply(weighted_average, column_proba_name,'absweight')
    
    
    title1 ='recoNjetsCat('+str(i-1)+') \n predicted by clf'
    title2 ='recoNjetsCat('+str(i-1)+') \n true from data'
   
    if (i==0) :
        print('perform efficiency (1- reco-proba) plot')
        plot_imshow([gb_proba,gb_freq],binBoundaries=defineBins,x_lab=x_var,y_lab=y_var,titles=['predicted (clf) reco eff',
                                                                    'true data reco eff'],effTag=True)
    else :
        plot_imshow([gb_proba,gb_freq],binBoundaries=defineBins, x_lab=x_var,y_lab=y_var,titles=[title1,title2])
 
    

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/swshare/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/swshare/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/swshare/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/swshare/anaconda/lib/python2.7/inspect.py", line 1049, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/swshare/anaconda/lib/python2.7/inspect.py", line 1009, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/swshare/anaconda/lib/python2.7/inspect.py", line 454, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "/swshare/anaconda/lib/python

IndexError: string index out of range

In [14]:
from matplotlib import ticker

#### Perform plots with realtive ratio of true and predicted probas:  $ \frac{\text{true} - \text{pred}}{\text{true}} $

In [16]:
N_recoNjets2p5Cats = 1

for i in np.arange(N_recoNjets2p5Cats) :
    column_proba_name = 'recoNjets2p5Cat_prob_'+str(i)
    
    gb_freq  = df.groupby([x_var+'Bin',y_var+'Bin']).apply(weight_freq,"recoNjets2p5Cat",(i-1),'absweight')
    gb_proba = df_test.groupby([x_var+'Bin',y_var+'Bin']).apply(weighted_average, column_proba_name,'absweight')
    
    if i == 0 :
        plotRelDiff_imshow([gb_proba,gb_freq],binBoundaries=defineBins,x_lab=x_var,y_lab=y_var,cmap=plt.cm.Oranges,effPlot=True)
    else :
        plotRelDiff_imshow([gb_proba,gb_freq],binBoundaries=defineBins,x_lab=x_var,y_lab=y_var,cmap=plt.cm.Oranges)
        

KeyboardInterrupt: 